1) For both Smith-Waterman and Needleman-Wunsch algorithms:
    a) What are the parameters and variables required for algorithm initialization, execution, and termination?
    b) What quantities are returned?
    c) What is the runtime complexity?
    
    Smith-Waterman:
        a) Parameters:
            Initialization:
                a) seq1: First protein sequence
                b) seq2: Second protein sequence
                c) smat: scoring matrix
            Execution:
                a) scores: matrix of size M x N that records scores of optimal alignment
                b) traceback: matrix of size M x N that records previous cell from which each cell was computed from
            Termination:
                a) result: optimal alignment string
        b) Returns: Optimal sub-alignment with positive scores
        c) Complexity: O(MxN)

    Needleman-Wunsch:
        a) Parameters:
            Initialization:
                a) seq1: First protein sequence
                b) seq2: Second protein sequence
                c) smat: scoring matrix
            Execution:
                a) scores: matrix of size M x N that records scores of optimal alignment
                b) traceback: matrix of size M x N that records previous cell from which each cell was computed from
            Termination:
                a) result: optimal alignment string
        b) Returns: Full alignment string of sequence 
        c) Complexity: O(MxN)

2) What functionalities in initialization, execution and termination are shared between these algorithms? Which are not shared?

    Initialization: Both methods initialize and M x N matrix where M is the length of seq1 and N is the length of seq2. However, in initializing the SW matrix, you set the first row and first column to 0. 
    
    Execution: Both methods share methods for calculating the score at a particular cell in the matrix. This entails taking the max of the three following scores:
        a) Gap from left to right
        b) Gap from top to bottom
        c) Match
    However, SW adds an extra option where you can reset to 0 if the score goes negative.
    
    Termination: Both methods have to use the traceback matrix to return an alignment. NW must start at the lower rightmost cell and trace back all the way to the start and return that string. SW needs to start at the max element in the matrix and trace back until a 0 is reached, then return the resulting string.

3) How does affine-gap based alignment differ from linear-gap alignment in terms of implementation?
    
    Affine gap alignment requires you to keep track of the gap status of the previous cell when computing the score of the current cell (gap start, match/mismatch, gap extension). To return the optimal alignment you need two extra score matrices, one for seq1 and one for seq2, which keep track of the max score of either opening or extending a gap at the current cell. For the final traceback you would need to interface between the original score matrix and these two extra matrices to get the alignment.

In [3]:
f = open("./sequences/prot-0004.fa").read()

In [15]:

f = open("./sequences/prot-0035.fa",'r').read()
f

'>d2hmqa_ 1.20.4.1.1\nGFPIPDPYCWDISFRTFYTIVDDEHKTLFNGILLLSQADNADHLNELRRCTGKHFLNEQQ\nLMQASQYAGYAEHKKAHDDFIHKLDTWDGDVTYAKNWLVNHIKTIDFKYRGKI\n'

In [9]:
f = open("./sequences/prot-0088.fa",'r').read()
f

'>d1glqa1 1.34.1.1.2\nYGKNQREAAQMDMVNDGVEDLRGKYVTLIYTNYENGKNDYVKALPGHLKPFETLLSQNQG\nGKAFIVGDQISFADYNLLDLLLIHQVLAPGCLDNFPLLSAYVARLSARPKIKAFLSSPEH\nVNRPINGNGKQ\n'

In [21]:
seq = []
with open("./sequences/prot-0004.fa" , 'r') as f:
    line = f.readline()
    if line.startswith(">"):
        continue
    seq.append(line)


SyntaxError: 'continue' not properly in loop (<ipython-input-21-a3a004a791c0>, line 8)

In [26]:
def read_sequence(file):
    f = open(file,'r')
    lines = f.readlines()[1:] # skip header
    seq = []
    for line in lines:
        seq += line.strip()
    return seq

In [27]:
seq1 =read_sequence("./sequences/prot-0004.fa")
seq1


['S',
 'L',
 'E',
 'A',
 'A',
 'Q',
 'K',
 'S',
 'N',
 'V',
 'T',
 'S',
 'S',
 'W',
 'A',
 'K',
 'A',
 'S',
 'A',
 'A',
 'W',
 'G',
 'T',
 'A',
 'G',
 'P',
 'E',
 'F',
 'F',
 'M',
 'A',
 'L',
 'F',
 'D',
 'A',
 'H',
 'D',
 'D',
 'V',
 'F',
 'A',
 'K',
 'F',
 'S',
 'G',
 'L',
 'F',
 'S',
 'G',
 'A',
 'A',
 'K',
 'G',
 'T',
 'V',
 'K',
 'N',
 'T',
 'P',
 'E',
 'M',
 'A',
 'A',
 'Q',
 'A',
 'Q',
 'S',
 'F',
 'K',
 'G',
 'L',
 'V',
 'S',
 'N',
 'W',
 'V',
 'D',
 'N',
 'L',
 'D',
 'N',
 'A',
 'G',
 'A',
 'L',
 'E',
 'G',
 'Q',
 'C',
 'K',
 'T',
 'F',
 'A',
 'A',
 'N',
 'H',
 'K',
 'A',
 'R',
 'G',
 'I',
 'S',
 'A',
 'G',
 'Q',
 'L',
 'E',
 'A',
 'A',
 'F',
 'K',
 'V',
 'L',
 'S',
 'G',
 'F',
 'M',
 'K',
 'S',
 'Y',
 'G',
 'G',
 'D',
 'E',
 'G',
 'A',
 'W',
 'T',
 'A',
 'V',
 'A',
 'G',
 'A',
 'L',
 'M',
 'G',
 'E',
 'I',
 'E',
 'P',
 'D',
 'M']

In [28]:
seq2 =read_sequence("./sequences/prot-0008.fa")
seq2


['A',
 'N',
 'K',
 'T',
 'R',
 'E',
 'L',
 'C',
 'M',
 'K',
 'S',
 'L',
 'E',
 'H',
 'A',
 'K',
 'V',
 'D',
 'T',
 'S',
 'N',
 'E',
 'A',
 'R',
 'Q',
 'D',
 'G',
 'I',
 'D',
 'L',
 'Y',
 'K',
 'H',
 'M',
 'F',
 'E',
 'N',
 'Y',
 'P',
 'P',
 'L',
 'R',
 'K',
 'Y',
 'F',
 'K',
 'S',
 'R',
 'E',
 'E',
 'Y',
 'T',
 'A',
 'E',
 'D',
 'V',
 'Q',
 'N',
 'D',
 'P',
 'F',
 'F',
 'A',
 'K',
 'Q',
 'G',
 'Q',
 'K',
 'I',
 'L',
 'L',
 'A',
 'C',
 'H',
 'V',
 'L',
 'C',
 'A',
 'T',
 'Y',
 'D',
 'D',
 'R',
 'E',
 'T',
 'F',
 'N',
 'A',
 'Y',
 'T',
 'R',
 'E',
 'L',
 'L',
 'D',
 'R',
 'H',
 'A',
 'R',
 'D',
 'H',
 'V',
 'H',
 'M',
 'P',
 'P',
 'E',
 'V',
 'W',
 'T',
 'D',
 'F',
 'W',
 'K',
 'L',
 'F',
 'E',
 'E',
 'Y',
 'L',
 'G',
 'K',
 'K',
 'T',
 'T',
 'L',
 'D',
 'E',
 'P',
 'T',
 'K',
 'Q',
 'A',
 'W',
 'H',
 'E',
 'I',
 'G',
 'R',
 'E',
 'F',
 'A',
 'K',
 'E',
 'I',
 'N',
 'K']

In [30]:
def test_fasta_io():
    file = "/Users/mtsui1/Documents/Classes/Algs/Project1/sequences/prot-0026.fa"
    seq = read_sequence(file)
    assert seq

In [36]:
seq = ["A", "B"]
assert seq

In [4]:
import numpy as np
import pandas as pd

In [39]:
a = np.zeros(shape=(5,2))
a.shape

(5, 2)

In [43]:
a[:,0] = 1
a[0,:] = 1

In [44]:
a

array([[1., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
scores = np.zeros(shape=(5,2))

In [21]:
scores

array([[1., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [20]:
scores[:,0] = 1
scores[0,:] = 1

In [23]:
for i in range(1,2):
    print(scores[i,1])

0.0


In [6]:
def read_score_matrix(smat):
    skiplines = 0
    with open(smat,'r') as fh:
        for line in fh:
            if line.startswith("#") | line.startswith("."):
                skiplines += 1
            else:
                break
    scores = pd.read_csv(smat, delim_whitespace=True, skiprows=skiplines)
    scores.set_index(scores.columns, inplace=True)
    return scores

In [25]:
smat = read_score_matrix("./scoring_matrices/BLOSUM50.mat")

In [11]:
f = open("./scoring_matrices/PAM250.mat")
f.readlines()

['. #\n',
 '# This matrix was produced by "pam" Version 1.0.6 [28-Jul-93]\n',
 '#\n',
 '# PAM 250 substitution matrix, scale = ln(2)/3 = 0.231049\n',
 '#\n',
 '# Expected score = -0.844, Entropy = 0.354 bits\n',
 '#\n',
 '# Lowest score = -8, Highest score = 17\n',
 '#\n',
 ' A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *\n',
 ' 2 -2  0  0 -2  0  0  1 -1 -1 -2 -1 -1 -3  1  1  1 -6 -3  0  0  0  0 -8\n',
 '-2  6  0 -1 -4  1 -1 -3  2 -2 -3  3  0 -4  0  0 -1  2 -4 -2 -1  0 -1 -8\n',
 ' 0  0  2  2 -4  1  1  0  2 -2 -3  1 -2 -3  0  1  0 -4 -2 -2  2  1  0 -8\n',
 ' 0 -1  2  4 -5  2  3  1  1 -2 -4  0 -3 -6 -1  0  0 -7 -4 -2  3  3 -1 -8\n',
 '-2 -4 -4 -5 12 -5 -5 -3 -3 -2 -6 -5 -5 -4 -3  0 -2 -8  0 -2 -4 -5 -3 -8\n',
 ' 0  1  1  2 -5  4  2 -1  3 -2 -2  1 -1 -5  0 -1 -1 -5 -4 -2  1  3 -1 -8\n',
 ' 0 -1  1  3 -5  2  4  0  1 -2 -3  0 -2 -5 -1  0  0 -7 -4 -2  3  3 -1 -8\n',
 ' 1 -3  0  1 -3 -1  0  5 -2 -3 -4 -2 -3 -5  0  1  0 -7 -5 -1  0  0 -1 -8\n',
 '-1  2  2  1 -3  3  1 -

In [12]:
scores = np.empty(shape=(5,6))


In [41]:
scores

array([[1., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [16]:
scores[0,0]

-2.6815615859885194e+154

In [38]:
smat.iloc[2:,2:]

,N,D,C,Q,E,G,H,I,L,K,...,P,S,T,W,Y,V,B,Z,X,*
N,7,2,-2,0,0,0,1,-3,-4,0,...,-2,1,0,-4,-2,-3,4,0,-1,-5
D,2,8,-4,0,2,-1,-1,-4,-4,-1,...,-1,0,-1,-5,-3,-4,5,1,-1,-5
C,-2,-4,13,-3,-3,-3,-3,-2,-2,-3,...,-4,-1,-1,-5,-3,-1,-3,-3,-2,-5
Q,0,0,-3,7,2,-2,1,-3,-2,2,...,-1,0,-1,-1,-1,-3,0,4,-1,-5
E,0,2,-3,2,6,-3,0,-4,-3,1,...,-1,-1,-1,-3,-2,-3,1,5,-1,-5
G,0,-1,-3,-2,-3,8,-2,-4,-4,-2,...,-2,0,-2,-3,-3,-4,-1,-2,-2,-5
H,1,-1,-3,1,0,-2,10,-4,-3,0,...,-2,-1,-2,-3,2,-4,0,0,-1,-5
I,-3,-4,-2,-3,-4,-4,-4,5,2,-3,...,-3,-3,-1,-3,-1,4,-4,-3,-1,-5
L,-4,-4,-2,-2,-3,-4,-3,2,5,-3,...,-4,-3,-1,-2,-1,1,-4,-3,-1,-5
K,0,-1,-3,2,1,-2,0,-3,-3,6,...,-1,0,-1,-3,-2,-3,0,1,-1,-5


In [36]:
smat.loc[seq1[1], seq2[1]]

-4

In [40]:
score = 5
gapx = 5
gapy =6
if score == gapx == gapy:
    print("hello")

In [65]:
yo = {"M":5, "IX":5, "IY":2}
best= max(yo, )

In [66]:
best

'M'

In [62]:
for key in yo.keys():
    if yo[key] <3:
        yo[key] = 0
        

SyntaxError: invalid syntax (<ipython-input-62-92dd95d2394c>, line 2)

In [83]:
m = max(yo.values())
items= [k for k,v in yo.items() if v == m]
{k: yo.get(k, None) for k in items}

{'M': 5, 'IX': 5}

In [82]:
yo[items]

TypeError: unhashable type: 'list'

In [84]:
yo["L"] = 2
yo

{'M': 5, 'IX': 5, 'IY': 2, 'L': 2}

In [86]:
def align(seq1, seq2):
    if len(seq1) == 0 | len(seq2) == 0:
        raise ValueError("Both sequences cannot be empty")
seq1 = ""
seq2 = ""
align(seq1,seq2)

ValueError: Both sequences cannot be empty

In [457]:
import numpy as np
import pandas as pd

class PairwiseAligner:
    def __init__(self, smat):
        self.smat = read_score_matrix(smat)
        self.scores = None
        self.gapX = None
        self.gapY = None
        self.traceback = None
        self.score = 0
        self.gap_open = -5
        self.gap_extend = -1

    '''
    Method that reads in a protein sequence from a fasta file
    Parameters:
        file: Path to fasta file
    Returns: list containing the sequence from the file
    '''
    def read_sequence(file):
        f = open(file,'r')
        lines = f.readlines()[1:] # skip header
        seq = []
        for line in lines:
            seq += line.strip()
        return seq

    '''
     Method that reads in score matrix 
     Parameters:
        smat: path to score matrix
     Returns: score matrix as a pandas df, where score lookup can be done
        as mat.loc["A","N"]
    '''
    def read_score_matrix(smat):
        skiplines = 0
        with open(smat,'r') as fh:
            for line in fh:
                if line.startswith("#") | line.startswith("."):
                    skiplines += 1
                else:
                    break
        scores = pd.read_csv(smat, delim_whitespace=True, skiprows=skiplines)
        scores.set_index(scores.columns, inplace=True)
        return scores

    def set_gap_open(self, gap_open):
        self.gap_open = gap_open
    
    def set_gap_extend(self, gap_extend):
        self.gap_extend = gap_extend
        
    def set_seqs(self, seq1, seq2):
        self.seq1 = read_sequence(seq1)
        self.seq2 = read_sequence(seq2)

    '''
     Align method: 
     Parameters:
        seq1: path to first sequence
        seq2: path to second sequence
        with_score: if true, returns the score of the alignment
            in addition to the alignments
        readable: if true, returns the two sequences with lines
            for readability
     Returns: returns seq1 and seq2 with best alignment
    '''
    def align(self, seq1, seq2, with_score=False, readable=False):
        if len(seq1) == 0 | len(seq2) == 0:
            raise ValueError("sequences cannot be empty")

        set_seqs(seq1, seq2)

        # initialize score matrix with seq1/seq2 
        init_mats()

        # loop through matrix, skipping boundary row/col
        for i in range(1,len(self.seq1)+1):
            for j in range(1,len(self.seq2)+1):
                fill_in(i,j)
        
        # traceback and return the score and alignment
        score, alignment = do_traceback()
        print(score)
        print(alignment)

class SmithWaterman(PairwiseAligner):
    def __init__(self, smat):
        super().__init__(smat)
        self.max = (0,0)

    '''
    Method to initialize the score-keeping and traceback matrices.
    seq1 is initialized on the rows, while seq2 on the columns
    Parameters:
    Returns: score, gap X, gap Y, and traceback matrices
    '''
    def init_mats(self):
        self.scores = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        self.traceback = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1),dtype=object)
        self.gapX = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        self.gapY = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        
        self.scores[:,0] = 0
        self.scores[0,:] = 0
        
        self.gapX[:,0] = 0
        self.gapX[0,:] = 0
        
        self.gapY[:,0] = 0
        self.gapY[0,:] = 0


    '''
    Fills in score in the scores matrix and arrow in the traceback
    for the current cell
    Parameters:
        i: row index
        j: col index
    Returns: score/arrows are filled in
    '''
    def fill_in(self, i, j):
        # calculate match score
        residue_score = self.smat.loc[self.seq1[i-1], self.seq2[j-1]]
        match = self.scores[i-1,j-1] + residue_score
        end_gapx = self.gapX[i-1,j-1] + residue_score
        end_gapy = self.gapY[i-1,j-1] + residue_score

        match_score = max(match, end_gapx, end_gapy)

        # calculate gap in X score
        gapx_open = self.scores[i,j-1] + self.gap_open + self.gap_extend
        gapx_extend = self.gapX[i,j-1] + self.gap_extend
        gapx_score = max(gapx_open, gapx_extend)

        # calculate gap in Y score
        gapy_open = self.scores[i-1,j] + self.gap_open + self.gap_extend
        gapy_extend = self.gapY[i-1,j] + self.gap_extend
        gapy_score = max(gapy_open, gapy_extend)

        score_dict = {"M":match_score, "Ix":gapx_score, "Iy":gapy_score}

        # set negative scores to 0
        for key in score_dict.keys():
            if score_dict[key] < 0:
                score_dict[key] = 0
        #print(score_dict)
        self.scores[i,j] = score_dict["M"]
        self.gapX[i,j] = score_dict["Ix"]
        self.gapY[i,j] = score_dict["Iy"]

        max_score = max(score_dict.values())

        # assign arrows (nonzero scores only)
        if max_score != 0:
            # for max score, get the cell that it came from
            # to resolve ties, a match is preferred, then X gap, then Y gap
            max_mats = [k for k,v in score_dict.items() if v == max_score]
            if "M" in max_mats:
                if max_score == match:
                    self.traceback[i,j] = "match"
                elif max_score == end_gapx:
                    self.traceback[i,j] = "endgapx"
                elif max_score == end_gapy:
                    self.traceback[i,j] = "endgapy"
            elif "Ix" in max_mats:
                if max_score == gapx_open:
                    self.traceback[i,j] = "opengapx"
                else:
                    self.traceback[i,j] = "extendgapx"
            elif "Iy" in max_mats:
                if max_score == gapy_open:
                    self.traceback[i,j] = "opengapy"
                else:
                    self.traceback[i,j] = "extendgapy"

        # update max if needed
        if max_score >= self.scores[self.max]:
            self.max = (i,j)


    def do_traceback(self):
        i = self.max[0]
        j = self.max[1]
        self.score = self.scores[i, j]
        
        # start traceback at max value
        current_val = self.scores[self.max[0], self.max[1]]
        str1 = ""
        str2 = ""
        while current_val > 0:
            if self.traceback[i,j] == "match" or self.traceback[i,j] == "endgapx" or self.traceback[i,j] == "endgapy":
                str1 = self.seq1[i-1] + str1
                str2 = self.seq2[j-1] + str2
                i = i-1
                j = j-1
                if self.traceback[i,j] == "match":
                    current_val = self.scores[i,j]
                elif self.traceback[i,j] == "endgapx":
                    current_val = self.gapX[i,j]
                elif self.traceback[i,j] == "endgapy":
                    current_val = self.gapY[i,j]
            elif self.traceback[i,j] == "opengapx":
                str1 = "-" + str1
                str2 = self.seq2[j-1] + str2
                j = j-1
                current_val = self.scores[i,j]
            elif self.traceback[i,j] == "extendgapx":
                str1 = "-" + str1
                str2 = self.seq2[j-1] + str2
                j = j-1
                current_val = self.gapX[i,j]
            elif self.traceback[i,j] == "opengapy":
                str1 = self.seq1[i-1] + str1
                str2 = "-" + str2
                i = i-1
                current_val = self.scores[i,j]
            elif self.traceback[i,j] == "extendgapy":
                str1 = self.seq1[i-1] + str1
                str2 = "-" + str2
                i = i-1
                current_val = self.gapY[i,j]
        
        print(current_val)
        alignment = str1 + '\n' + str2
        return self.score, alignment

class NeedlemanWunsch(PairwiseAligner):
    def __init__(self, smat):
        super().__init__(smat)
        
    def init_mats(self):
        self.scores = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        self.traceback = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1),dtype=object)
        self.gapX = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        self.gapY = np.empty(shape=(len(self.seq1)+1,len(self.seq2)+1))
        
        self.scores[0,0] = 0
        self.scores[0,1:] = float("-inf")
        self.scores[1:,0] = float("-inf")
        
        self.gapX[:,0] = list(range(len(self.gapX[:,0])))
        self.gapX[:,0] = [idx*self.gap_extend+self.gap_open for idx in self.gapX[:,0]]
        self.gapX[0,1:] = float("-inf")
        
        self.gapY[1:,0] = float("-inf")
        self.gapY[0,:] = list(range(len(self.gapY[0,:])))
        self.gapY[0,:] = [idx*self.gap_extend+self.gap_open for idx in self.gapY[0,:]]
        
    '''
    Fills in score in the scores matrix and arrow in the traceback
    for the current cell
    Parameters:
        i: row index
        j: col index
    Returns: score/arrows are filled in
    '''
    def fill_in(self, i, j):
        # calculate match score
        residue_score = self.smat.loc[self.seq1[i-1], self.seq2[j-1]]
        match = self.scores[i-1,j-1] + residue_score
        end_gapx = self.gapX[i-1,j-1] + residue_score
        end_gapy = self.gapY[i-1,j-1] + residue_score

        match_score = max(match, end_gapx, end_gapy)
        #print(match_score)
        # calculate gap in X score
        gapx_open = self.scores[i-1,j] + self.gap_open + self.gap_extend
        gapx_extend = self.gapX[i-1,j] + self.gap_extend
        gapx_score = max(gapx_open, gapx_extend)
        #print(gapx_open,gapx_extend)
        # calculate gap in Y score
        gapy_open = self.scores[i,j-1] + self.gap_open + self.gap_extend
        gapy_extend = self.gapY[i,j-1] + self.gap_extend
        gapy_score = max(gapy_open, gapy_extend)
        #print(gapy_open, gapy_extend)
        #print(gapy_score)
        score_dict = {"M":match_score, "Ix":gapx_score, "Iy":gapy_score}

        #print(score_dict)
        self.scores[i,j] = score_dict["M"]
        self.gapX[i,j] = score_dict["Ix"]
        self.gapY[i,j] = score_dict["Iy"]

        max_score = max(score_dict.values())

        # assign arrows

        # for max score, get the cell that it came from
        # to resolve ties, a match is preferred, then X gap, then Y gap
        max_mats = [k for k,v in score_dict.items() if v == max_score]
        if "M" in max_mats:
            if max_score == match:
                self.traceback[i,j] = "match"
            elif max_score == end_gapx:
                self.traceback[i,j] = "endgapx"
            elif max_score == end_gapy:
                self.traceback[i,j] = "endgapy"
        elif "Ix" in max_mats:
            if max_score == gapx_open:
                self.traceback[i,j] = "opengapx"
            else:
                self.traceback[i,j] = "extendgapx"
        elif "Iy" in max_mats:
            if max_score == gapy_open:
                self.traceback[i,j] = "opengapy"
            else:
                self.traceback[i,j] = "extendgapy"

    def do_traceback(self):
        i = len(self.scores[:,0]) - 1
        j = len(self.scores[0,:]) - 1
        
        # score is the biggest value in the bottom right cell
        self.score = max(self.scores[i,j], self.gapX[i,j], self.gapY[i,j])
        
        str1 = ""
        str2 = ""
        
        # traceback until you reach 0,0
        while i > 0 or j > 0:
            print(i,j)
            if self.traceback[i,j] == "match" or self.traceback[i,j] == "endgapx" or self.traceback[i,j] == "endgapy":
                str1 = self.seq1[i-1] + str1
                str2 = self.seq2[j-1] + str2
                i = i-1
                j = j-1
            elif self.traceback[i,j] == "opengapx":
                str1 = "-" + str1
                str2 = self.seq2[j-1] + str2
                j = j-1
            elif self.traceback[i,j] == "extendgapx":
                str1 = "-" + str1
                str2 = self.seq2[j-1] + str2
                j = j-1
            elif self.traceback[i,j] == "opengapy":
                str1 = self.seq1[i-1] + str1
                str2 = "-" + str2
                i = i-1
            elif self.traceback[i,j] == "extendgapy":
                str1 = self.seq1[i-1] + str1
                str2 = "-" + str2
                i = i-1

        
        alignment = str1 + '\n' + str2
        return self.score, alignment

In [466]:
nw = NeedlemanWunsch("./scoring_matrices/BLOSUM50.mat")

In [459]:
nw.seq1 = "BTN"
nw.seq2 = "BT"
nw.init_mats()



In [368]:
len(nw.gapY[:,0])

3

In [460]:
# loop through matrix, skipping boundary row/col
for i in range(1,len(nw.seq1)+1):
    for j in range(1,len(nw.seq2)+1):
        nw.fill_in(i,j)

In [383]:
sw.scores[11,4]

IndexError: index 11 is out of bounds for axis 0 with size 5

In [461]:
nw.scores

array([[  0., -inf, -inf],
       [-inf,   5.,  -6.],
       [-inf,  -6.,  10.],
       [-inf,  -3.,  -1.]])

In [462]:
nw.gapX


array([[ -5., -inf, -inf],
       [ -6., -inf, -inf],
       [ -7.,  -1., -12.],
       [ -8.,  -2.,   4.]])

In [463]:
nw.gapY

array([[ -5.,  -6.,  -7.],
       [-inf, -inf,  -1.],
       [-inf, -inf, -12.],
       [-inf, -inf,  -9.]])

In [464]:
nw.traceback

array([[None, None, None],
       [None, 'match', 'opengapy'],
       [None, 'opengapx', 'match'],
       [None, 'extendgapx', 'opengapx']], dtype=object)

In [465]:
score, alignment = nw.do_traceback()
print(score)
print(alignment)

3 2
3 1
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0


3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0


3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0


3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0


3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0


KeyboardInterrupt: 

In [404]:
float("-inf") + 2

-inf

In [237]:
sw.scores[sw.max[0], sw.max[1]]

17.0

In [212]:
end = False
i = 0
while not end:
    i+=1
    print(i)
    if i == 3:
        end = True


1
2
3


In [240]:
str1 = "HHHHHHLBTEG"
str2 = "LNEG"

alignment = str1 + '\n' + str2
print(alignment)


HHHHHHLBTEG
LNEG


In [273]:
current_val

NameError: name 'current_val' is not defined

In [333]:
float("-inf") + float("inf")

nan

In [336]:
list(range(5))*-1 + -3

TypeError: can only concatenate list (not "int") to list

In [341]:
l = list(range(5))
l = [element*2+1 for element in l]
l

[1, 3, 5, 7, 9]

In [369]:
i = 2
j = 2
while i != 0 or j != 0:
    print(i,j)
    i-= 1
    j-= 1

2 2
1 1
